# Economic Indicator Analysis
## Multi-Agent Economic Forecasting System - Notebook 2

**Objective**: Perform comprehensive economic indicator analysis using statistical tools and the Economic Analyst agent.

### What You'll Learn:
- GDP growth trend analysis and pattern recognition
- Business cycle identification and phase detection
- Economic health assessment using multiple indicators
- Anomaly detection in economic data
- Using the Economic Analyst agent for automated analysis

## 1. Setup and Data Preparation

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio

# Add src to path
sys.path.append('../src')

# Import our components
from tools.statistical_tools import StatisticalTools
from agents.economic_analyst import EconomicAnalystAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# Setup visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries and components imported successfully")

In [ ]:
# Initialize components
stat_tools = StatisticalTools()

# Initialize model for agents
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

model = Gemini(
    model="gemini-2.0-flash-exp",
    retry_options=retry_config
)

# Initialize Economic Analyst Agent
economic_analyst = EconomicAnalystAgent(model)

print("🤖 Economic Analyst Agent initialized")
print("🛠️ Available analysis tools:")
for tool in economic_analyst.agent.tools:
    print(f"   - {tool.name}")

## 2. Load and Prepare Economic Data

In [ ]:
# Load sample economic data (from previous notebook or generate synthetic)
def generate_sample_economic_data():
    """Generate sample economic data for analysis"""
    dates = pd.date_range(start='2010-01-01', end='2024-12-31', freq='Q')
    np.random.seed(42)

    # Generate realistic GDP data with trend, seasonality, and cycles
    trend = np.linspace(100, 180, len(dates))
    seasonal = 8 * np.sin(2 * np.pi * np.arange(len(dates)) / 4)

    # Add business cycles
    cycle_length = 32  # 8-year cycles
    cycles = 15 * np.sin(2 * np.pi * np.arange(len(dates)) / cycle_length)

    noise = np.random.normal(0, 3, len(dates))
    gdp_data = trend + seasonal + cycles + noise

    # Create some anomalies
    anomaly_indices = [10, 25, 40]
    for idx in anomaly_indices:
        gdp_data[idx] += np.random.normal(20, 5)

    return pd.DataFrame({
        'TimePeriod': dates,
        'DataValue': gdp_data,
        'Series': 'GDP'
    })

# Generate sample data
economic_data = generate_sample_economic_data()
print(f"📊 Sample economic data generated: {len(economic_data)} quarters")
print(f"📅 Date range: {economic_data['TimePeriod'].min()} to {economic_data['TimePeriod'].max()}")
economic_data.head()

## 3. Growth Trend Analysis

In [ ]:
# Perform comprehensive growth trend analysis
print("📈 Analyzing GDP Growth Trends...")
growth_analysis = stat_tools.analyze_growth_trends(economic_data)

if growth_analysis['status'] == 'success':
    print("✅ Growth Trend Analysis Results:")
    print(f"   Current Growth Rate: {growth_analysis.get('current_growth_rate', 'N/A'):.2f}%")
    print(f"   Average Growth Rate: {growth_analysis.get('average_growth_rate', 'N/A'):.2f}%")
    print(f"   Trend Direction: {growth_analysis.get('trend_direction', 'N/A')}")
    print(f"   Trend Strength: {growth_analysis.get('trend_strength', 'N/A'):.3f}")
    print(f"   Volatility: {growth_analysis.get('volatility', 'N/A'):.2f}%")
    print(f"   Seasonal Strength: {growth_analysis.get('seasonal_strength', 'N/A'):.3f}")
else:
    print("❌ Growth analysis failed")
    print(growth_analysis)

In [ ]:
# Visualize growth trends
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Original time series
ax1.plot(economic_data['TimePeriod'], economic_data['DataValue'],
         linewidth=2, color='#1f77b4', label='GDP')
ax1.set_title('GDP Time Series', fontsize=14, fontweight='bold')
ax1.set_ylabel('GDP Value')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Growth rates
growth_rates = economic_data['DataValue'].pct_change() * 100
ax2.plot(economic_data['TimePeriod'][1:], growth_rates[1:],
         linewidth=2, color='#2ca02c', label='Quarterly Growth %')
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)
ax2.set_title('Quarterly Growth Rates', fontsize=14, fontweight='bold')
ax2.set_ylabel('Growth Rate (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Rolling statistics
rolling_mean = economic_data['DataValue'].rolling(window=4).mean()
rolling_std = economic_data['DataValue'].rolling(window=4).std()
ax3.plot(economic_data['TimePeriod'], rolling_mean,
         linewidth=2, color='#ff7f0e', label='4-Quarter Moving Avg')
ax3.fill_between(economic_data['TimePeriod'],
                rolling_mean - rolling_std,
                rolling_mean + rolling_std,
                alpha=0.3, color='#ff7f0e', label='±1 Std Dev')
ax3.set_title('Trend with Volatility', fontsize=14, fontweight='bold')
ax3.set_ylabel('GDP Value')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Annual growth
annual_growth = economic_data['DataValue'].pct_change(4) * 100
ax4.plot(economic_data['TimePeriod'][4:], annual_growth[4:],
         linewidth=2, color='#d62728', label='Annual Growth %')
ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7)
ax4.set_title('Annual Growth Rates', fontsize=14, fontweight='bold')
ax4.set_ylabel('Annual Growth (%)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Business Cycle Analysis

In [ ]:
# Identify business cycles
print("🔄 Analyzing Business Cycles...")
cycle_analysis = stat_tools.identify_business_cycles(economic_data)

if cycle_analysis['status'] == 'success':
    print("✅ Business Cycle Analysis Results:")
    print(f"   Current Phase: {cycle_analysis.get('current_phase', 'N/A')}")
    print(f"   Peaks Identified: {len(cycle_analysis.get('peaks', []))}")
    print(f"   Troughs Identified: {len(cycle_analysis.get('troughs', []))}")
    print(f"   Cycle Duration: {cycle_analysis.get('cycle_duration', {})}")
    print(f"   Cycle Amplitude: {cycle_analysis.get('amplitude', 'N/A'):.2f}")
else:
    print("❌ Business cycle analysis failed")
    print(cycle_analysis)

In [ ]:
# Visualize business cycles
if cycle_analysis['status'] == 'success' and 'cycle_component' in cycle_analysis:

    fig = make_subplots(rows=2, cols=1,
                       subplot_titles=('GDP with Business Cycle Phases', 'Cycle Component'),
                       vertical_spacing=0.1)

    # Plot 1: GDP with cycle phases
    fig.add_trace(
        go.Scatter(x=economic_data['TimePeriod'], y=economic_data['DataValue'],
                  mode='lines', name='GDP', line=dict(color='#1f77b4', width=3)),
        row=1, col=1
    )

    # Highlight peaks and troughs
    peaks = cycle_analysis.get('peaks', [])
    troughs = cycle_analysis.get('troughs', [])

    if peaks:
        peak_dates = [economic_data['TimePeriod'].iloc[i] for i in peaks if i < len(economic_data)]
        peak_values = [economic_data['DataValue'].iloc[i] for i in peaks if i < len(economic_data)]
        fig.add_trace(
            go.Scatter(x=peak_dates, y=peak_values,
                      mode='markers', name='Peaks',
                      marker=dict(color='red', size=10, symbol='triangle-down')),
            row=1, col=1
        )

    if troughs:
        trough_dates = [economic_data['TimePeriod'].iloc[i] for i in troughs if i < len(economic_data)]
        trough_values = [economic_data['DataValue'].iloc[i] for i in troughs if i < len(economic_data)]
        fig.add_trace(
            go.Scatter(x=trough_dates, y=trough_values,
                      mode='markers', name='Troughs',
                      marker=dict(color='green', size=10, symbol='triangle-up')),
            row=1, col=1
        )

    # Plot 2: Cycle component
    cycle_component = cycle_analysis.get('cycle_component', [])
    if cycle_component:
        fig.add_trace(
            go.Scatter(x=economic_data['TimePeriod'], y=cycle_component,
                      mode='lines', name='Cycle Component',
                      line=dict(color='#ff7f0e', width=2)),
            row=2, col=1
        )

    fig.update_layout(height=600, title_text="Business Cycle Analysis", showlegend=True)
    fig.show()
else:
    print("❌ Insufficient data for business cycle visualization")

## 5. Economic Indicator Calculation

In [ ]:
# Calculate comprehensive economic indicators
print("📊 Calculating Economic Indicators...")
indicators = stat_tools.calculate_indicators(economic_data)

if indicators['status'] == 'success':
    print("✅ Key Economic Indicators:")
    indicator_data = indicators['indicators']

    # Create a summary table
    indicator_summary = pd.DataFrame([
        ['Current Value', f"{indicator_data.get('current_value', 0):.2f}"],
        ['Mean', f"{indicator_data.get('mean', 0):.2f}"],
        ['Standard Deviation', f"{indicator_data.get('std_dev', 0):.2f}"],
        ['Coefficient of Variation', f"{indicator_data.get('cv', 0):.2f}%"],
        ['Recent Growth (1Y)', f"{indicator_data.get('recent_growth', 0):.2f}%"],
        ['Long-term Growth', f"{indicator_data.get('long_term_growth', 0):.2f}%"],
        ['Volatility', f"{indicator_data.get('volatility', 0):.2f}%"],
        ['Max Drawdown', f"{indicator_data.get('max_drawdown', 0):.2f}%"],
        ['Trend Strength', f"{indicator_data.get('trend_strength', 0):.3f}"],
        ['Stationary', f"{indicator_data.get('stationary', False)}"],
        ['Business Cycle Position', f"{indicator_data.get('business_cycle_position', 'N/A')}"],
        ['Momentum', f"{indicator_data.get('momentum', 0):.2f}%"],
    ], columns=['Indicator', 'Value'])

    print(indicator_summary.to_string(index=False))

    print(f"\n💡 Interpretation: {indicators.get('interpretation', 'No interpretation available')}")
else:
    print("❌ Indicator calculation failed")

## 6. Anomaly Detection

In [ ]:
# Detect anomalies in economic data
print("🔍 Performing Anomaly Detection...")

# Try different detection methods
methods = ['zscore', 'iqr']
anomaly_results = {}

for method in methods:
    print(f"\nTesting {method.upper()} method:")
    anomalies = stat_tools.detect_anomalies(economic_data, method=method, threshold=2.0)

    if anomalies['status'] == 'success':
        print(f"   Anomalies found: {anomalies['anomaly_count']}")
        print(f"   Anomaly percentage: {anomalies['anomaly_percentage']:.2f}%")
        anomaly_results[method] = anomalies
    else:
        print(f"   ❌ {method} method failed")

# Display detected anomalies
if anomaly_results:
    best_method = list(anomaly_results.keys())[0]
    anomalies = anomaly_results[best_method]['anomalies']

    if anomalies:
        print(f"\n📋 Detected Anomalies ({best_method.upper()} method):")
        for i, anomaly in enumerate(anomalies[:5]):  # Show first 5
            print(f"   {i+1}. Date: {anomaly['date']}, Value: {anomaly['value']:.2f}, "
                  f"Deviation: {anomaly['deviation']:.2f}σ, Type: {anomaly['type']}")
    else:
        print("\n✅ No significant anomalies detected")

In [ ]:
# Visualize anomalies
if anomaly_results and anomaly_results[best_method]['anomalies']:
    fig = go.Figure()

    # Plot normal data
    normal_dates = []
    normal_values = []
    anomaly_dates = []
    anomaly_values = []

    all_dates = economic_data['TimePeriod'].tolist()
    all_values = economic_data['DataValue'].tolist()
    anomaly_info = {anom['date']: anom for anom in anomaly_results[best_method]['anomalies']}

    for date, value in zip(all_dates, all_values):
        date_str = date.strftime('%Y-%m-%d')
        if date_str in anomaly_info:
            anomaly_dates.append(date)
            anomaly_values.append(value)
        else:
            normal_dates.append(date)
            normal_values.append(value)

    # Add traces
    fig.add_trace(go.Scatter(
        x=normal_dates, y=normal_values,
        mode='lines', name='Normal Data',
        line=dict(color='#1f77b4', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=anomaly_dates, y=anomaly_values,
        mode='markers', name='Anomalies',
        marker=dict(color='red', size=10, symbol='x')
    ))

    fig.update_layout(
        title='Economic Data with Detected Anomalies',
        xaxis_title='Time Period',
        yaxis_title='GDP Value',
        showlegend=True
    )

    fig.show()
else:
    print("❌ No anomalies to visualize")

## 7. Using Economic Analyst Agent

In [ ]:
# Test the Economic Analyst Agent
async def test_economic_analyst():
    """Test the economic analyst agent's analysis capabilities"""

    print("🧪 Testing Economic Analyst Agent...")

    # Convert data to list of dictionaries for the agent
    data_dict = economic_data.to_dict('records')

    # Test growth trend analysis
    print("\n🔸 Testing growth trend analysis...")
    growth_result = await economic_analyst.analyze_growth_trends(data_dict)
    if growth_result['status'] == 'success':
        print(f"   Trend: {growth_result.get('trend', 'N/A')}")
        print(f"   Confidence: {growth_result.get('confidence', 0):.2f}")

    # Test indicator calculation
    print("\n🔸 Testing indicator calculation...")
    indicator_result = await economic_analyst.calculate_economic_indicators(data_dict)
    if indicator_result['status'] == 'success':
        print(f"   Indicators calculated: {len(indicator_result.get('indicators', {}))}")

    # Test business cycle analysis
    print("\n🔸 Testing business cycle analysis...")
    cycle_result = await economic_analyst.identify_business_cycles(data_dict)
    if cycle_result['status'] == 'success':
        print(f"   Current phase: {cycle_result.get('current_phase', 'N/A')}")

    # Test anomaly detection
    print("\n🔸 Testing anomaly detection...")
    anomaly_result = await economic_analyst.detect_anomalies(data_dict)
    if anomaly_result['status'] == 'success':
        print(f"   Anomalies found: {anomaly_result.get('anomaly_count', 0)}")

# Run the agent tests
await test_economic_analyst()

## 8. Comprehensive Economic Health Assessment

In [ ]:
# Create a comprehensive economic health dashboard
def assess_economic_health(growth_analysis, indicators, cycle_analysis, anomalies):
    """Provide overall economic health assessment"""

    health_score = 0
    factors = []

    # Growth factor (0-40 points)
    if growth_analysis['status'] == 'success':
        current_growth = growth_analysis.get('current_growth_rate', 0)
        if current_growth > 3:
            health_score += 40
            factors.append("Strong growth (>3%)")
        elif current_growth > 0:
            health_score += 25
            factors.append("Positive growth")
        else:
            health_score += 10
            factors.append("Negative growth")

    # Volatility factor (0-20 points)
    if indicators['status'] == 'success':
        volatility = indicators['indicators'].get('volatility', 10)
        if volatility < 2:
            health_score += 20
            factors.append("Low volatility")
        elif volatility < 5:
            health_score += 15
            factors.append("Moderate volatility")
        else:
            health_score += 5
            factors.append("High volatility")

    # Business cycle factor (0-20 points)
    if cycle_analysis['status'] == 'success':
        current_phase = cycle_analysis.get('current_phase', '')
        if current_phase == 'expansion':
            health_score += 20
            factors.append("Expansion phase")
        elif current_phase == 'stable':
            health_score += 15
            factors.append("Stable phase")
        else:
            health_score += 5
            factors.append("Contraction phase")

    # Anomaly factor (0-20 points)
    if anomalies['status'] == 'success':
        anomaly_percentage = anomalies.get('anomaly_percentage', 0)
        if anomaly_percentage < 5:
            health_score += 20
            factors.append("Few anomalies")
        elif anomaly_percentage < 10:
            health_score += 15
            factors.append("Moderate anomalies")
        else:
            health_score += 5
            factors.append("Many anomalies")

    # Determine health status
    if health_score >= 80:
        status = "Excellent"
        color = "green"
    elif health_score >= 60:
        status = "Good"
        color = "blue"
    elif health_score >= 40:
        status = "Moderate"
        color = "orange"
    else:
        status = "Poor"
        color = "red"

    return {
        'score': health_score,
        'status': status,
        'color': color,
        'factors': factors
    }

# Perform health assessment
if all(analysis['status'] == 'success' for analysis in [growth_analysis, indicators, cycle_analysis, anomalies]):
    health_assessment = assess_economic_health(growth_analysis, indicators, cycle_analysis, anomalies)

    print("🏥 ECONOMIC HEALTH ASSESSMENT")
    print("=" * 40)
    print(f"Overall Score: {health_assessment['score']}/100")
    print(f"Status: {health_assessment['status']}")
    print(f"\nKey Factors:")
    for factor in health_assessment['factors']:
        print(f"  ✓ {factor}")
else:
    print("❌ Cannot perform health assessment - some analyses failed")

## 9. Summary and Next Steps

In [ ]:
print("🎯 NOTEBOOK 2 SUMMARY")
print("=" * 50)

# Summary of analyses performed
analyses_performed = []
if growth_analysis['status'] == 'success':
    analyses_performed.append("Growth Trend Analysis")
if indicators['status'] == 'success':
    analyses_performed.append("Economic Indicators")
if cycle_analysis['status'] == 'success':
    analyses_performed.append("Business Cycle Analysis")
if 'anomalies' in locals() and anomalies['status'] == 'success':
    analyses_performed.append("Anomaly Detection")

print(f"✅ Analyses completed: {len(analyses_performed)}")
for analysis in analyses_performed:
    print(f"   • {analysis}")

print(f"\n📊 Data analyzed: {len(economic_data)} quarters")
print(f"📅 Period: {economic_data['TimePeriod'].min().strftime('%Y-%m')} to {economic_data['TimePeriod'].max().strftime('%Y-%m')}")

print("\n🔜 Next Steps:")
print("   1. Proceed to Notebook 3: Forecasting Model Development")
print("   2. Use real BEA data instead of synthetic data")
print("   3. Expand analysis to multiple economic indicators")
print("   4. Implement automated monitoring with the Economic Analyst Agent")

print("\n💡 Production Insights:")
print("   - The Economic Analyst Agent can automate routine analysis tasks")
print("   - Statistical tools provide consistent, reproducible analysis")
print("   - Business cycle analysis helps with strategic planning")
print("   - Anomaly detection identifies data quality issues and economic shocks")